In [77]:
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia

In [2]:
Gaia.MAIN_GAIA_TABLE = "gaiaedr3.gaia_source" # Select early Data Release 3

In [171]:
class ICRS2GCC():

    def __init__(self, source_id):
        self.source_id = source_id
    
    def query_gaia_object(self):
        """
        Function that returns all gaia information for the object with id source_id
        
        Returns:
            Gaia information (astropy table): astrometric and photometric information for the object with id source_id
        """
        query = """SELECT * FROM gaiaedr3.gaia_source 
        WHERE source_id = {Gaia_ID}"""
        query = query.format(Gaia_ID=self.source_id)
        job = Gaia.launch_job_async(query)
        return job.get_results()
    
    def matrix_cov(self):
        """
        Function that returns the covariance matrix using uncertainties and correlations coefficient from Gaia
        
        Returns:
            matrix covariance (numpy array): matrix covariance
        """
        r = self.query_gaia_object()
        # empty matrix to build the covariance matrix
        c = np.empty((6, 6), dtype=float)
        c[0, 0] = r['ra_error']*r['ra_error']
        c[0, 1] = c[1, 0] = r['ra_error']*r['dec_error']*r['ra_dec_corr']
        c[0, 2] = c[2, 0] = r['ra_error']*r['parallax_error']*r['ra_parallax_corr']
        c[0, 3] = c[3, 0] = r['ra_error']*r['pmra_error']*r['ra_pmra_corr']
        c[0, 4] = c[4, 0] = r['ra_error']*r['pmdec_error']*r['ra_pmdec_corr']
        c[0, 5] = c[5, 0] = 0
        c[1, 1] = r['dec_error']*r['dec_error']
        c[1, 2] = c[2, 1] = r['dec_error']*r['parallax_error']*r['dec_parallax_corr']
        c[1, 3] = c[3, 1] = r['dec_error']*r['pmra_error']*r['dec_pmra_corr']
        c[1, 4] = c[4, 1] = r['dec_error']*r['pmdec_error']*r['dec_pmdec_corr']
        c[1, 5] = c[5, 1] = 0
        c[2, 2] = r['parallax_error']*r['parallax_error']
        c[2, 3] = c[3, 2] = r['parallax_error']*r['pmra_error']*r['parallax_pmra_corr']
        c[2, 4] = c[4, 2] = r['parallax_error']*r['pmdec_error']*r['parallax_pmdec_corr']
        c[2, 5] = c[5, 2] = 0
        c[3, 3] = r['pmra_error']*r['pmra_error']
        c[3, 4] = c[4, 3] = r['pmra_error']*r['pmdec_error']*r['pmra_pmdec_corr']
        c[3, 5] = c[5, 3] = 0
        c[4, 4] = r['pmdec_error']*r['pmdec_error']
        c[4, 5] = c[5, 4] = 0
        c[5, 5] = r['dr2_radial_velocity_error']*r['dr2_radial_velocity_error']
        return c
#    def icrs_sampling(self):

In [25]:
#def query_gaia_object(source_id):
#    """
#    
#    """
#    query = """SELECT * FROM gaiaedr3.gaia_source 
#    WHERE source_id = {Gaia_ID}"""
#    query = query.format(Gaia_ID=source_id)
#    job = Gaia.launch_job(query)
#    return job.get_results()

In [172]:
obj = ICRS2GCC(4667368899326729856)
r = obj.matrix_cov()
r

INFO: Query finished. [astroquery.utils.tap.core]


array([[ 9.54406787e-05,  8.07745346e-06,  1.46465482e-05,
        -1.83462234e-05, -1.47798009e-05,  0.00000000e+00],
       [ 8.07745346e-06,  9.08365109e-05,  2.41361377e-05,
        -1.35102800e-06, -1.68362167e-05,  0.00000000e+00],
       [ 1.46465482e-05,  2.41361377e-05,  1.10947549e-04,
         3.21730076e-05, -2.69259599e-05,  0.00000000e+00],
       [-1.83462234e-05, -1.35102800e-06,  3.21730076e-05,
         1.77398950e-04,  3.24402681e-05,  0.00000000e+00],
       [-1.47798009e-05, -1.68362167e-05, -2.69259599e-05,
         3.24402681e-05,  1.77991737e-04,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  3.22557235e+00]])

In [123]:
#obj.source_id = 4667335913977929728
#r = obj.query_gaia_object()
#r

INFO: Query finished. [astroquery.utils.tap.core]


solution_id,designation,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,parallax_error,parallax_over_error,pm,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr,ra_parallax_corr,ra_pmra_corr,ra_pmdec_corr,dec_parallax_corr,dec_pmra_corr,dec_pmdec_corr,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr,astrometric_n_obs_al,astrometric_n_obs_ac,astrometric_n_good_obs_al,astrometric_n_bad_obs_al,astrometric_gof_al,astrometric_chi2_al,astrometric_excess_noise,astrometric_excess_noise_sig,astrometric_params_solved,astrometric_primary_flag,nu_eff_used_in_astrometry,pseudocolour,pseudocolour_error,ra_pseudocolour_corr,dec_pseudocolour_corr,parallax_pseudocolour_corr,pmra_pseudocolour_corr,pmdec_pseudocolour_corr,astrometric_matched_transits,visibility_periods_used,astrometric_sigma5d_max,matched_transits,new_matched_transits,matched_transits_removed,ipd_gof_harmonic_amplitude,ipd_gof_harmonic_phase,ipd_frac_multi_peak,ipd_frac_odd_win,ruwe,scan_direction_strength_k1,scan_direction_strength_k2,scan_direction_strength_k3,scan_direction_strength_k4,scan_direction_mean_k1,scan_direction_mean_k2,scan_direction_mean_k3,scan_direction_mean_k4,duplicated_source,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_flux_over_error,phot_g_mean_mag,phot_bp_n_obs,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_flux_over_error,phot_bp_mean_mag,phot_rp_n_obs,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_flux_over_error,phot_rp_mean_mag,phot_bp_n_contaminated_transits,phot_bp_n_blended_transits,phot_rp_n_contaminated_transits,phot_rp_n_blended_transits,phot_proc_mode,phot_bp_rp_excess_factor,bp_rp,bp_g,g_rp,dr2_radial_velocity,dr2_radial_velocity_error,dr2_rv_nb_transits,dr2_rv_template_teff,dr2_rv_template_logg,dr2_rv_template_fe_h,l,b,ecl_lon,ecl_lat
,,,,yr,deg,mas,deg,mas,mas,mas,,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,,,,,,,mas,,,,1 / um,1 / um,1 / um,,,,,,,,mas,,,,,deg,,,,,,,,deg,deg,deg,deg,,,electron / s,electron / s,,mag,,electron / s,electron / s,,mag,,electron / s,electron / s,,mag,,,,,,,mag,mag,mag,km / s,km / s,,K,log(cm.s**-2),dex,deg,deg,deg,deg
int64,object,int64,int64,float64,float64,float32,float64,float32,float64,float32,float32,float32,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,int16,float32,float32,float32,float32,int16,bool,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,float32,int16,int16,int16,float32,float32,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,bool,int16,float64,float32,float32,float32,int16,float64,float32,float32,float32,int16,float64,float32,float32,float32,int16,int16,int16,int16,int16,float32,float32,float32,float32,float32,float32,int16,float32,float32,float32,float64,float64,float64,float64
1636042515805110273,Gaia EDR3 4667335913977929728,4667335913977929728,1591815672,2016.0,54.39207400346642,0.019150086,-68.68499964893623,0.019367898,1.21830593036908,0.01906784,63.89323,18.107489,4.35147610520748,0.02400905,17.57685369873509,0.022759268,0.10612504,0.08568943,-0.34366313,-0.27086183,0.0754462,-0.25639552,-0.25372684,0.1018322,-0.23831023,0.1551066,303,0,301,2,21.050278,1175.0875,0.17704599,26.554602,31,False,1.5311195,--,--,--,--,--,--,--,35,27,0.0367375,38,17,0,0.025795206,73.736145,0,0,1.978536,0.16971408,0.048153892,0.045082685,0.05960808,-6.0223927,67.054436,27.120165,6.023081,False,308,87674.67439104567,18.848673,4651.504,13.330181,35,45049.9264558907,40.827145,1103.4308,13.704307,32,61053.05726004629,33.608616,1816.5895,12.783627,0,0,0,0,0,1.2101897,0.92068005,0.37412548,0.54655457,--,--,0,--,--,--,284.05906554142723,-41.833071175379196,334.8366085447379,-76.4773089156761


In [151]:
r.colnames

['solution_id',
 'designation',
 'source_id',
 'random_index',
 'ref_epoch',
 'ra',
 'ra_error',
 'dec',
 'dec_error',
 'parallax',
 'parallax_error',
 'parallax_over_error',
 'pm',
 'pmra',
 'pmra_error',
 'pmdec',
 'pmdec_error',
 'ra_dec_corr',
 'ra_parallax_corr',
 'ra_pmra_corr',
 'ra_pmdec_corr',
 'dec_parallax_corr',
 'dec_pmra_corr',
 'dec_pmdec_corr',
 'parallax_pmra_corr',
 'parallax_pmdec_corr',
 'pmra_pmdec_corr',
 'astrometric_n_obs_al',
 'astrometric_n_obs_ac',
 'astrometric_n_good_obs_al',
 'astrometric_n_bad_obs_al',
 'astrometric_gof_al',
 'astrometric_chi2_al',
 'astrometric_excess_noise',
 'astrometric_excess_noise_sig',
 'astrometric_params_solved',
 'astrometric_primary_flag',
 'nu_eff_used_in_astrometry',
 'pseudocolour',
 'pseudocolour_error',
 'ra_pseudocolour_corr',
 'dec_pseudocolour_corr',
 'parallax_pseudocolour_corr',
 'pmra_pseudocolour_corr',
 'pmdec_pseudocolour_corr',
 'astrometric_matched_transits',
 'visibility_periods_used',
 'astrometric_sigma5d_max